<h3>Packages Required</h3>


In [1]:
import string
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from nltk.util import flatten

<h3>Reading Manufacturers Data</h3> 



In [2]:
Complete_Manufacturers = pd.read_csv('Manufacturers_list.csv')
Complete_Manufacturers["Capabilities_Overall"].head()

0    ['Boring Services', 'Broaching Services', 'Cen...
1    ['Abrasive Compounds / Media', 'Acrylic', 'Aer...
2    ['Assemblies', 'Aircraft Parts', 'Aluminum', '...
3    ['Analytical Services', 'Automation Systems', ...
4    ['Additive Manufacturing', 'Aerospace Componen...
Name: Capabilities_Overall, dtype: object

In [3]:
Complete_Manufacturers.shape

(13190, 8)

<h3>Selecting the "State" and "Manufacturing Category" in which we are looking for manufacturers
    </h3>

In [4]:
Input_State = 'CA'
Input_Text  = ["COVID-19 Response"]

In [5]:
Manufacturers_By_State = Complete_Manufacturers[Complete_Manufacturers["Manufacturer_state"] == Input_State]

<h3>Total Number of Results for Selected state and Category</h3>

In [6]:
Manufacturers_By_State.head

<bound method NDFrame.head of                                               Name  \
2                        LeFiell Manufacturing Co.   
24                       Garland's Precision, Inc.   
38                   Wilkinson Manufacturing, Inc.   
61                      Kinetic Machining Services   
65                Custom Tube & Manufacturing Inc.   
86                               Caliber Engraving   
99     International Polymer Solutions, (iPolymer)   
113                    Binder Metal Products, Inc.   
124              BTM - Beartech Manufacturing, Inc   
125                    U.S. Titanium Industry Inc.   
142                     Sun Fast International LLC   
151                     Impro Industries USA, Inc.   
176                           Aranda Tooling, Inc.   
188                                ABC Sheet Metal   
205                            Applied Engineering   
211                  New Technology Plastics, Inc.   
223                    Serra Laser & Waterjet, Inc  

<h3>Selecting only the Capabilities of manufacturers</h3>

In [7]:
Manufacturer_Capabilities = Manufacturers_By_State["Capabilities_Overall"]

<h3>Function to clean and  Normalize the Text</h3>

In [8]:
def Text_Normalization(Manufacturer_Capabilities,Input_Text):
    list_for_matrix =[]
    input_list = []
    for i in range(len(Manufacturer_Capabilities)):
        text = "".join(Manufacturer_Capabilities.iloc[i])
        text = text.lower()
        text = text.strip()
        text = "".join([char for char in text if char not in string.punctuation])
        text = re.sub("r(\d)","", text)
        text = re.sub(r'\(.*?\)', '', text)
        tokens = word_tokenize(text)
        list_for_matrix.append(" ".join(tokens))
        
    for j in range(len(Input_Text)):
        text1 = "".join(Input_Text[j])       
        text1 = text1.lower()       
        text1 = text1.strip()     
        text1 = "".join([char for char in text1 if char not in string.punctuation])      
        text1 = re.sub("r(\d)","", text1)        
        text1 = re.sub(r'\(.*?\)', '', text1)       
        tokens1 = word_tokenize(text1)
        input_list.append(" ".join(tokens1))
        input_list =(input_list)
        print(input_list)
    list_for_matrix.insert(0,input_list)
    list_for_matrix =flatten(list_for_matrix)

    return list_for_matrix

# Text_Normalization(Manufacturer_Capabilities,Input_Text)

<h3>Function to convert the Text to matrix for each manufacturer using the TfidfVectorizer</h3>

In [9]:
def Vectorizer (Normalized_data):
    vectorizer = TfidfVectorizer(stop_words='english')
    matrix = vectorizer.fit_transform(Normalized_data)
    print(matrix.toarray())
    Input_matrix = matrix[0]
    new_matrix = matrix[1:]
    print(Input_matrix.shape)
    print(new_matrix.shape)
    return new_matrix,Input_matrix


<h3>converting the InputText and Manufacturer_Capabilities to matrix and storing them into X and Input_matrix</h3>

In [10]:
Normalized_data = Text_Normalization(Manufacturer_Capabilities,Input_Text)
X = Vectorizer (Normalized_data)[0]
Input_matrix = Vectorizer (Normalized_data)[1]

['covid19 response']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1, 1117)
(1495, 1117)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1, 1117)
(1495, 1117)


<h3>Calculating the similarity score for Input and Manufacturing_Capabilities matrix</h3>

In [11]:
cosine_score = cosine_similarity(Input_matrix, X)
company_score = cosine_score.tolist()
company_score= flatten(company_score)
len(company_score)

1495

<h3>storing the Manufacturer scores and ID's in sorting order in a list</h3>

In [12]:
data_frame =pd.DataFrame()
data_frame['company_score'] = company_score
data_frame['ID'] = data_frame.index
data_frame.head()


,company_score,ID
0,0.000000,0
1,0.158775,1
2,0.000000,2
3,0.318701,3
4,0.000000,4


In [13]:
DataFrame = data_frame.sort_values(by ='company_score',ascending=False)
print(DataFrame.head(10))
list_of_final_manufactureres = DataFrame["ID"].head(10)


      company_score    ID
381        0.445438   381
318        0.429004   318
445        0.425167   445
463        0.390867   463
546        0.386685   546
428        0.375450   428
690        0.365071   690
611        0.362529   611
47         0.353399    47
1302       0.353399  1302


<h3>Displaying the Manufacturers based on the Similarity Score</h3>

In [14]:
Suggested_manufacturers = Manufacturers_By_State.iloc[list_of_final_manufactureres]
Suggested_manufacturers

,Name,Address,Manufacturer_state,Number,URL,Employees,Description,Capabilities_Overall
3098,Lazar Machining,"San Carlos, CA 94070",CA,650-591-6415,http://www.lazarmfg.com,1-9,Company Description by Thomasnet\nCustom manuf...,"['COVID-19 Response', 'Machining', 'Milling Se..."
2677,California Drop Forge,"Los Angeles, CA 90012",CA,323-221-1134,http://www.californiadrop.com,50-99,Company Description by Thomasnet\nCustom manuf...,"['COVID-19 Response', 'Forging Services', 'Mac..."
3580,"Markall, Inc.","Santa Ana, CA 92704",CA,877-431-6756,http://www.markallinc.com,Not Available,Company Description by Thomasnet\nCustom manuf...,"['Assemblies', 'COVID-19 Response', 'Machining..."
3706,JCM Corp.,"Ontario, CA 91761",CA,909-923-3730,http://jcmcorp.com/,100-199,Company Description by Thomasnet\nCustom manuf...,"['Assemblies', 'COVID-19 Response', 'Machining..."
4431,Marsway Machining Inc,"Oceanside, CA 92056",CA,760-631-0872,http://www.marsway.com,Not Available,Company Description by Thomasnet\nCustom manuf...,"['COVID-19 Response', 'Fabrication Services', ..."
3432,TOMI Engineering,"Santa Ana, CA 92707",CA,714-556-1474,https://tomiengineering.com/,50-99,Company Description by Thomasnet\nCustom manuf...,"['Casting Services', 'COVID-19 Response', 'For..."
5569,Mayfly Manufacturing,"Saratoga, CA 95070",CA,408-472-5350,https://mayflymfg.com,Not Available,"Company Description by Thomasnet\nDesign, engi...","['COVID-19 Response', 'Machining', 'Medical Co..."
4955,J & T Manufacturing and Engineering,"San Diego, CA 92120",CA,619-265-5951,http://www.jtmfg.com,Not Available,Company Description by Thomasnet\nCustom manuf...,"['COVID-19 Response', 'Cutting Services', 'Mac..."
422,"Swiss Machine Products, Inc.","Anaheim, CA 92806",CA,833-979-1677,http://swissmachine.com/,Not Available,Company Description by Thomasnet\nCustom manuf...,"['COVID-19 Response', 'Machining', 'Medical Co..."
11066,"Vanderhulst Associates, Inc.","Santa Clara, CA 95054",CA,408-727-1313,http://www.vanderhulst.com,10-49,Company Description by Thomasnet\nMachine work...,"['COVID-19 Response', 'Machining', 'Medical Co..."
